# IPO Performance
- Retrieves historical price performance
- Uses Yahoo Finance for historical price data
 - https://finance.yahoo.com/

In [1]:
import numpy as np
import pandas as pd

import nasdaq

from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# params
start_date = '2018-01-01'
ipo_performance_csv = 'IPO Performance.csv'

# Create NASDAQ IPO List

In [3]:
df_symbols = nasdaq.get_ipo_list(start_date)
print('symbols', df_symbols.shape[0])

date range: odict_keys(['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06'])
symbols 107


In [4]:
df_symbols.head()

,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced
Symbol,,,,,,,
MTECU,MTECH ACQUISITION CORP,MTECU,NASDAQ Capital,10.0,5000000,50000000.0,2018-01-30
AGS,"PLAYAGS, INC.",AGS,NYSE,16.0,10250000,164000000.0,2018-01-26
ARMO,"ARMO BIOSCIENCES, INC.",ARMO,NASDAQ Global Select,17.0,7529412,128000004.0,2018-01-26
SLDB,SOLID BIOSCIENCES INC.,SLDB,NASDAQ Global Select,16.0,7812500,125000000.0,2018-01-26
TORC,"RESTORBIO, INC.",TORC,NASDAQ Global Select,15.0,5666667,85000005.0,2018-01-26


In [5]:
#load previous to save time
df_prev = pd.read_csv(ipo_performance_csv)
df_prev.index = df_prev['Symbol']

#reduce by previously done
df_symbols = df_symbols.loc[df_symbols.index.difference(df_prev.index)]

EmptyDataError: No columns to parse from file

# Add Historical Price Performance

- Use open prices only (retial traders won't get best price)

In [6]:
#target variables and trading days range
targets = {
    '1D' : 1,
    '1W' : 5,
    '1M' : 20,
    '3M' : 60,
}

In [7]:
#add targets
for t in targets.keys():
    df_symbols[t] = np.NaN

#calculate targets    
counter = 0    
    
for x in df_symbols['Symbol']:
    try:
        counter += 1
        print('\n( ' + str(counter) + ' / ' + str(df_symbols.shape[0]) + ' ) ' + str(x))

        #get historical prices
        df = pdr.get_data_yahoo(x, start=start_date)

        #add targets
        for t in targets.keys():
            s = df.pct_change(periods=targets[t]).dropna().head(1)['Open']
            if df.head(1).index[0] == df_symbols.at[x, 'Date Priced'] and s.shape[0] > 0 and abs(s[0]) > 0:
                df_symbols.at[x, t] = s[0]
            else:
                df_symbols.at[x, t] = np.NaN
                
        #store
        #df_full = pd.concat([df_symbols, df_prev], axis=0)
        #df_full.dropna(inplace=True)        
        #df_full.to_csv('IPO Performance.csv', index=False)                
    except Exception as e:
        print(x, e)     


( 1 / 107 ) MTECU
[*********************100%***********************]  1 of 1 downloaded

( 2 / 107 ) AGS
[*********************100%***********************]  1 of 1 downloaded

( 3 / 107 ) ARMO
[*********************100%***********************]  1 of 1 downloaded

( 4 / 107 ) SLDB
[*********************100%***********************]  1 of 1 downloaded

( 5 / 107 ) TORC
[*********************100%***********************]  1 of 1 downloaded

( 6 / 107 ) EYEN
[*********************100%***********************]  1 of 1 downloaded

( 7 / 107 ) GPAQU
[*********************100%***********************]  1 of 1 downloaded

( 8 / 107 ) GTES
[*********************100%***********************]  1 of 1 downloaded

( 9 / 107 ) MNLO
[*********************100%***********************]  1 of 1 downloaded

( 10 / 107 ) PAGS
[*********************100%***********************]  1 of 1 downloaded

( 11 / 107 ) NINE
[*********************100%***********************]  1 of 1 downloaded

( 12 / 107 ) ADT
[**********

[*********************100%***********************]  1 of 1 downloaded

( 93 / 107 ) BCML
[*********************100%***********************]  1 of 1 downloaded

( 94 / 107 ) CBLK
[*********************100%***********************]  1 of 1 downloaded

( 95 / 107 ) INSP
[*********************100%***********************]  1 of 1 downloaded

( 96 / 107 ) UBX
[*********************100%***********************]  1 of 1 downloaded

( 97 / 107 ) XSPL
[*********************100%***********************]  1 of 1 downloaded

( 98 / 107 ) PRT
[*********************100%***********************]  1 of 1 downloaded

( 99 / 107 ) AVLR
[*********************100%***********************]  1 of 1 downloaded

( 100 / 107 ) NEW
[*********************100%***********************]  1 of 1 downloaded

( 101 / 107 ) VRCA
[*********************100%***********************]  1 of 1 downloaded

( 102 / 107 ) USX
[*********************100%***********************]  1 of 1 downloaded

( 103 / 107 ) CHRA
[********************

In [8]:
print('done', df_symbols.shape[0], 'prev', df_prev.shape[0])
df_symbols.head(3)

NameError: name 'df_prev' is not defined

In [9]:
df_full = pd.concat([df_symbols], axis=0)
df_full.dropna(inplace=True)        
df_full.to_csv('IPO Performance.csv', index=False) 